<a href="https://colab.research.google.com/github/mrh-rakib/WASSA-2023-empathy/blob/main/main_EMP_transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [ ]:
# from evaluation import pearsonr, calculate_pearson

In [2]:
from google.colab import drive
mount_path = '/content/drive'
drive.mount(mount_path)

%cd $mount_path"/MyDrive/WASSA2023"

raw_data = pd.read_csv("./essay_article_texts.csv", index_col=0)

Mounted at /content/drive


In [60]:
raw_data.head(2)

,article,essay,speaker_id,gender,education,race,age,income,speaker_number,essay_id,empathy,distress
0,"A month after Hurricane Matthew, 800,000 Haiti...",It breaks my heart to see people living in tho...,30,1.0,6.0,3.0,37.0,40000.0,1,1,6.714286,6.714286
1,"A month after Hurricane Matthew, 800,000 Haiti...",I wonder why there aren't more people trying t...,19,1.0,6.0,2.0,32.0,35000.0,1,2,5.857143,6.000000


# Empathy

In [7]:
chosen_data = raw_data[['article', 'essay', 'empathy']]

In [96]:
!pip install transformers datasets sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [92]:
import transformers as trf
from datasets import Dataset
import torch
from tqdm.auto import tqdm

In [61]:
hugging_dataset = Dataset.from_pandas(chosen_data, preserve_index=False)
hugging_dataset = hugging_dataset.train_test_split(test_size = 0.2)

In [11]:
# hugging_dataset

In [11]:
# hugging_dataset['train']['essay'][:5]

In [145]:
# checkpoint = "bert-base-uncased"
checkpoint = "bhadresh-savani/bert-base-uncased-emotion"
# checkpoint = "distilbert-base-uncased"
# checkpoint = "cardiffnlp/twitter-roberta-base-sentiment-latest"

In [146]:
tokeniser = trf.AutoTokenizer.from_pretrained(checkpoint)

In [147]:
#padding="longest" can be deferred to do dynamic padding
def tokenise(sentence):
  return tokeniser(sentence["essay"], sentence["article"], truncation=True) 
  # return tokeniser(sentence["essay"], sentence["article"], padding="max_length", max_length=514, truncation=True)   #for Cardiff-emotion one
  # return tokeniser(sentence["essay"], truncation=True) 
  # return tokeniser(sentence["article"], sentence["essay"], truncation=True) 

In [148]:
tokenised_hugging_dataset = hugging_dataset.map(tokenise, batched=True)

# tokenised_hugging_dataset

Map:   0%|          | 0/623 [00:00<?, ? examples/s]

Map:   0%|          | 0/156 [00:00<?, ? examples/s]

In [150]:
# checking length after tokenisation

length = []
for i in range(tokenised_hugging_dataset['train'].num_rows):
  length.append(len(tokenised_hugging_dataset['train']['input_ids'][i]))

print(f"Lengths: {length}")

Lengths: [512, 512, 512, 512, 512, 512, 512, 130, 442, 512, 512, 512, 512, 238, 512, 239, 512, 512, 512, 305, 512, 444, 512, 512, 512, 350, 205, 512, 512, 512, 512, 512, 512, 452, 512, 512, 512, 512, 314, 500, 512, 512, 345, 512, 512, 512, 512, 512, 512, 512, 512, 272, 512, 512, 512, 512, 512, 447, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 247, 436, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 318, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 282, 512, 512, 512, 512, 512, 512, 512, 451, 246, 512, 512, 512, 512, 512, 285, 512, 351, 512, 512, 512, 512, 512, 307, 445, 512, 512, 512, 460, 512, 512, 512, 512, 314, 512, 512, 494, 512, 512, 512, 512, 512, 512, 512, 512, 512, 229, 405, 512, 512, 317, 512, 271, 360, 512, 512, 512, 512, 278, 512, 512, 512, 512, 512, 512, 512, 512, 198, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 259, 512, 219, 512, 

In [151]:
tokenised_hugging_dataset = tokenised_hugging_dataset.remove_columns(["article","essay"]) # no longer required as encoding done
tokenised_hugging_dataset = tokenised_hugging_dataset.rename_column("empathy", "labels") # as huggingface requires
tokenised_hugging_dataset = tokenised_hugging_dataset.with_format("torch")

# tokenised_hugging_dataset

In [153]:
BATCH_SIZE = 8
LEARNING_RATE = 5e-5
NUM_EPOCH = 3

In [155]:
data_collator = trf.DataCollatorWithPadding(tokenizer = tokeniser)

train_dataloader = torch.utils.data.DataLoader(
    tokenised_hugging_dataset["train"], shuffle=True, batch_size=BATCH_SIZE, collate_fn=data_collator
)

test_dataloader = torch.utils.data.DataLoader(
    tokenised_hugging_dataset["test"], batch_size=BATCH_SIZE, collate_fn=data_collator
)

## Prediction model

In [157]:
# prediction_model = trf.AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=1)
prediction_model = trf.AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=1, ignore_mismatched_sizes=True)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bhadresh-savani/bert-base-uncased-emotion and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([6, 768]) in the checkpoint and torch.Size([1, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([6]) in the checkpoint and torch.Size([1]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
opt = torch.optim.AdamW(prediction_model.parameters(), lr=LEARNING_RATE)

training_steps = NUM_EPOCH * len(train_dataloader)
lr_scheduler = trf.get_scheduler(
    "linear",
    optimizer=opt,
    num_warmup_steps=0,
    num_training_steps=training_steps
)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
prediction_model.to(device)

device

In [ ]:
# criterion = torch.nn.MSELoss()

progress_bar = tqdm(range(training_steps))

prediction_model.train()
for epoch in range(NUM_EPOCH):
  epoch_loss = 0
  num_batches = 0
  for batch in train_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    outputs = prediction_model(**batch)
    loss = outputs.loss
    # loss = criterion(outputs.logits, batch['labels'])
    loss.backward()

    opt.step()
    lr_scheduler.step()
    opt.zero_grad()
    progress_bar.update(1)

    epoch_loss += loss.item()
    num_batches += 1

  avg_epoch_loss = epoch_loss / num_batches
  print(f"Epoch {epoch}: average loss = {avg_epoch_loss}")

## Hyperparameter tuning

In [52]:
!pip install ray

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.6/58.6 MB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 65.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.5/468.5 KB 47.8 MB/s eta 0:00:00


In [54]:
from ray import tune

In [ ]:
def train(config):
  prediction_model = trf.AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=1)

  learning_rate = config["learning_rate"]
  num_epochs = config["num_epochs"]
  batch_size = config["batch_size"]
  momentum = config["momentum"]

  opt = torch.optim.AdamW(prediction_model.parameters(), lr=learning_rate, momentum=momentum)
  # lr_scheduler = torch.optim.lr_scheduler.StepLR(opt, step_size=1, gamma=0.1)
  lr_scheduler = trf.get_scheduler(
    "linear",
    optimizer=opt,
    num_warmup_steps=0,
    num_training_steps=training_steps
  )
  # criterion = torch.nn.MSELoss()

  train_dataloader = torch.utils.data.DataLoader(
    tokenised_hugging_dataset["train"], shuffle=True, batch_size=batch_size, collate_fn=data_collator
    )
  
  prediction_model.train()
  for epoch in range(num_epochs):
    epoch_loss = 0
    num_batches = 0
    for batch in train_dataloader:
      batch = {k: v.to(device) for k, v in batch.items()}
      outputs = empathy_prediction(**batch)
      loss = outputs.loss 
      # loss = criterion(outputs.logits, batch['labels'])

      loss.backward()    
      opt.step()
      lr_scheduler.step()
      opt.zero_grad()
      progress_bar.update(1)

      epoch_loss += loss.item()
      num_batches += 1

    avg_epoch_loss = epoch_loss / num_batches
    tune.report(loss=avg_epoch_loss)

config = {
    "learning_rate": tune.loguniform(6e-5, 2e-5),
    "num_epochs": tune.choice([3, 5, 7]),
    "batch_size": tune.choice([4, 8, 16]),
    "momentum": tune.uniform(0.1, 0.9)
}

scheduler = tune.schedulers.ASHAScheduler(
    max_t = training_steps,
    grace_period=1,
    reduction_factor=2
)

analysis = tune.run(
    train,
    resources_per_trial={"cpu": 1, "gpu": 0.5},
    config=config,
    num_samples=10,
    scheduler=scheduler,
    progress_reporter=tune.CLIReporter(),
    local_dir="./ray_results"
)

best_trial = analysis.get_best_trial("loss", "min", "last")
print(f"Best trial config: {best_trial.config}")

## Evaluation

In [77]:
prediction_model.eval()

predictions = []

for batch in test_dataloader:
  batch = {k: v.to(device) for k, v in batch.items()}
  with torch.no_grad():
    outputs = prediction_model(**batch)
    
  batch_pred = [item for sublist in outputs.logits.tolist() for item in sublist]  #convert 2D list to 1D
  predictions.append(batch_pred)

y_pred = [item for sublist in predictions for item in sublist]  #convert batch-wise 2D list to 1D

In [ ]:
# prediction_model.save_pretrained("model")

In [78]:
y_true = hugging_dataset["test"]["empathy"]

In [46]:
# y_pred

In [62]:
# y_true

In [79]:
pearsonr(y_true,y_pred)

0.6287

In [29]:
from math import sqrt
to_round = 4

def pearsonr(x, y):
	"""
	Calculates a Pearson correlation coefficient. 
	"""

	assert len(x) == len(y), 'Prediction and gold standard does not have the same length...'

	xm = sum(x)/len(x)
	ym = sum(y)/len(y)

	xn = [k-xm for k in x]
	yn = [k-ym for k in y]

	r = 0 
	r_den_x = 0
	r_den_y = 0
	for xn_val, yn_val in zip(xn, yn):
		r += xn_val*yn_val
		r_den_x += xn_val*xn_val
		r_den_y += yn_val*yn_val

	r_den = sqrt(r_den_x*r_den_y)

	if r_den:
		r = r / r_den
	else:
		r = 0

	# Presumably, if abs(r) > 1, then it is only some small artifact of floating
	# point arithmetic.
	r = max(min(r, 1.0), -1.0)

	return round(r,to_round)

def calculate_pearson(gold, prediction):
	"""
	gold/prediction are a list of lists [ emp pred , distress pred ]
	"""

	# converting to float
	gold = [float(k) for k in gold]
	prediction = [float(k) for k in prediction]

	return pearsonr(gold, prediction)

# Training by Huggingface API

In [ ]:
from transformers import Trainer, TrainingArguments
from sklearn.metrics import mean_squared_error

In [56]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    rmse = mean_squared_error(labels, predictions, squared=False)
    return {"rmse": rmse}

In [65]:
training_args = TrainingArguments(output_dir="empathy-transformer",
                                  logging_strategy="epoch",
                                  evaluation_strategy="epoch",
                                  per_device_train_batch_size=16,
                                  per_device_eval_batch_size=16,
                                  num_train_epochs=3,
                                  # learning_rate=2e-5,
                                  save_total_limit=2,
                                  save_strategy='no',
                                  load_best_model_at_end=False)

trainer = Trainer(
    model=empathy_prediction,
    args=training_args,
    train_dataset=tokenised_hugging_dataset["train"],
    eval_dataset=tokenised_hugging_dataset["test"],
    data_collator=data_collator,
    tokenizer=tokeniser,
    compute_metrics=compute_metrics
)

In [66]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Rmse
1,3.415700,3.094043,1.758989
2,2.160800,2.747570,1.657580
3,1.413400,2.890623,1.700183


TrainOutput(global_step=117, training_loss=2.329951457488231, metrics={'train_runtime': 30.014, 'train_samples_per_second': 62.271, 'train_steps_per_second': 3.898, 'total_flos': 76837223949486.0, 'train_loss': 2.329951457488231, 'epoch': 3.0})

In [67]:
raw_pred, _, _ = trainer.predict(tokenised_hugging_dataset["test"])